In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import multiprocessing as mp
from langdetect import detect
from tqdm import tqdm

from pandarallel import pandarallel

import re
import ast

ModuleNotFoundError: No module named 'bs4'

In [21]:
links = pd.read_pickle("/dlabdata1/lugeon/official_links.pkl")
links.head()

,page_id,title,link_url_normalised
0,6632119,Gladstone Pottery Museum,www.stokemuseums.org.uk/visit/gpm
1,35732799,Vicky Shu,www.facebook.com/mademoiselle.vickyshu
2,23407708,5th Venice International Film Festival,www.labiennale.org/en/cinema
3,45623538,Greenpeace Energy,www.greenpeace-energy.de
4,7292565,Michael Lee (musician),www.myspace.com/michaelleedrummer


In [22]:
topics = pd.read_json('/dlabdata1/tmp/enwiki.labeled_article_items.json', lines=True)

In [14]:
topics.head(1)

,article_pid,wp_templates,article_revid,qid,sitelinks,title,talk_revid,talk_pid,taxo_labels
0,18951386.0,"[WikiProject Objectivism, WikiProject Novels, ...",926765055.0,Q374098,"{'la': 'Atlas Shrugged', 'sv': 'Och världen sk...",Atlas Shrugged,911346471,128,"[Culture.Media.Books, History and Society.Poli..."


In [23]:
topics.shape

(5926244, 9)

In [24]:
topics_red = topics[['article_pid', 'qid', 'taxo_labels']]
topics_red.head(1)

,article_pid,qid,taxo_labels
0,18951386.0,Q374098,"[Culture.Media.Books, History and Society.Poli..."


In [17]:
topics_red.to_csv('/dlabdata1/lugeon/wikitopics.gz', compression='gzip')

In [25]:
links.head(1)

,page_id,title,link_url_normalised
0,6632119,Gladstone Pottery Museum,www.stokemuseums.org.uk/visit/gpm


In [26]:
topics_red.head(1)

,article_pid,qid,taxo_labels
0,18951386.0,Q374098,"[Culture.Media.Books, History and Society.Poli..."


In [27]:
topics_red = topics_red[topics_red.article_pid.notnull()]

In [28]:
topics_red = topics_red.astype({'article_pid': int})
links = links.astype({'page_id': int})

In [29]:
merged = links.merge(topics_red, left_on='page_id', right_on='article_pid')

In [30]:
merged.head(1)

,page_id,title,link_url_normalised,article_pid,qid,taxo_labels
0,6632119,Gladstone Pottery Museum,www.stokemuseums.org.uk/visit/gpm,6632119,Q5566347,"[Culture.Visual arts.Architecture, Culture.Vis..."


In [31]:
merged.shape

(503056, 6)

In [35]:
def tokcat(cats):
    catlist = [field.split('.') for field in cats]
    cat1 = np.array([cat[0] for cat in catlist if len(cat) > 0])
    cat2 = np.array([cat[1] for cat in catlist if len(cat) > 1])
    cat3 = np.array([cat[2] for cat in catlist if len(cat) > 2])
    return np.unique(cat1).tolist(), np.unique(cat2).tolist(), np.unique(cat3).tolist()

In [36]:
merged['tokcat'] = merged.apply(lambda row: tokcat(row.taxo_labels), axis=1)

In [37]:
merged['cat1'] = merged.apply(lambda row: row.tokcat[0], axis=1)
merged['cat2'] = merged.apply(lambda row: row.tokcat[1], axis=1)
merged['cat3'] = merged.apply(lambda row: row.tokcat[2], axis=1)

In [38]:
merged

,page_id,title,link_url_normalised,article_pid,qid,taxo_labels,tokcat,cat1,cat2,cat3
0,6632119,Gladstone Pottery Museum,www.stokemuseums.org.uk/visit/gpm,6632119,Q5566347,"[Culture.Visual arts.Architecture, Culture.Vis...","([Culture], [Visual arts], [Architecture, Visu...",[Culture],[Visual arts],"[Architecture, Visual arts*]"
1,35732799,Vicky Shu,www.facebook.com/mademoiselle.vickyshu,35732799,Q4251749,"[Geography.Regions.Asia.Southeast Asia, Geogra...","([Culture, Geography], [Biography, Regions], [...","[Culture, Geography]","[Biography, Regions]","[Asia, Biography*, Women]"
2,23407708,5th Venice International Film Festival,www.labiennale.org/en/cinema,23407708,Q1606388,"[Culture.Media.Films, Culture.Media.Media*]","([Culture], [Media], [Films, Media*])",[Culture],[Media],"[Films, Media*]"
3,45623538,Greenpeace Energy,www.greenpeace-energy.de,45623538,Q1544765,"[History and Society.Education, Geography.Regi...","([Geography, History and Society], [Business a...","[Geography, History and Society]","[Business and economics, Education, Regions]",[Europe]
4,7292565,Michael Lee (musician),www.myspace.com/michaelleedrummer,7292565,Q433281,"[Culture.Media.Music, Culture.Media.Media*, Cu...","([Culture], [Biography, Media], [Biography*, M...",[Culture],"[Biography, Media]","[Biography*, Media*, Music]"
...,...,...,...,...,...,...,...,...,...,...
503051,15655778,Ghoul (band),www.creepsylvania.com,15655778,Q5557479,"[Culture.Media.Music, Culture.Media.Media*, Cu...","([Culture], [Biography, Media], [Biography*, M...",[Culture],"[Biography, Media]","[Biography*, Media*, Music]"
503052,16292465,Audrey Wood,audreywood.com,16292465,Q4820195,"[Geography.Regions.Americas.North America, Cul...","([Culture, Geography], [Biography, Literature,...","[Culture, Geography]","[Biography, Literature, Regions]","[Americas, Biography*, Women]"
503053,13090805,Bentleigh West Primary School,www.bentleighwestps.vic.edu.au,13090805,Q4890442,"[History and Society.Education, Geography.Regi...","([Geography, History and Society], [Education,...","[Geography, History and Society]","[Education, Politics and government, Regions]",[Oceania]
503054,11105025,California Department of Finance,www.dof.ca.gov,11105025,Q5020418,[Geography.Regions.Americas.North America],"([Geography], [Regions], [Americas])",[Geography],[Regions],[Americas]


In [39]:
allcat1 = pd.Series([c_ for c in merged.cat1 for c_ in c])
allcat1.value_counts()

Geography              380775
Culture                237989
History and Society    149533
STEM                    46325
dtype: int64

In [40]:
allcat2 = pd.Series([c_ for c in merged.cat2 for c_ in c])
allcat2.value_counts()

Regions                    380155
Media                      100931
Biography                   97726
Education                   53101
Business and economics      48285
STEM*                       46325
Sports                      36357
Politics and government     32568
Geographical                30846
Visual arts                 24130
Transportation              19324
Philosophy and religion     15187
Literature                  14738
Technology                  13740
Society                     13699
Engineering                 10177
Computing                    9441
Internet culture             8708
Medicine & Health            6377
Food and drink               6013
Military and warfare         4776
History                      3848
Biology                      3268
Performing arts              3086
Earth and environment        1956
Libraries & Information      1923
Space                        1897
Linguistics                   777
Physics                       572
Chemistry     

In [41]:
allcat2.unique().shape

(31,)

In [42]:
allcat3 = pd.Series([c_ for c in merged.cat3 for c_ in c])
allcat3.value_counts()

Americas            149940
Europe              126222
Media*              100931
Biography*           97726
Asia                 86561
Music                30654
Visual arts*         24130
Television           22925
Radio                20069
Oceania              19662
Women                17231
Architecture         15149
Entertainment        10355
Africa                8175
Software              6444
Films                 5153
Video games           3469
Comics and Anime      2948
Fashion               2587
Books                 1238
dtype: int64

In [43]:
merged = merged[['page_id', 'title', 'link_url_normalised', 'taxo_labels', 'cat1', 'cat2', 'cat3']]

In [44]:
merged.head(1)

,page_id,title,link_url_normalised,taxo_labels,cat1,cat2,cat3
0,6632119,Gladstone Pottery Museum,www.stokemuseums.org.uk/visit/gpm,"[Culture.Visual arts.Architecture, Culture.Vis...",[Culture],[Visual arts],"[Architecture, Visual arts*]"


In [45]:
merged.rename(columns={'link_url_normalised': 'link', 'taxo_labels': 'fullcat'}, inplace=True)

/dlabdata1/lugeon/lugeon-env/lib/python3.8/site-packages/pandas/core/frame.py:4295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [46]:
merged.to_csv('/dlabdata1/lugeon/wikilinks.gz', compression='gzip')

In [1]:
df = pd.read_csv("/dlabdata1/lugeon/wikilinks.gz", index_col=0)
df.shape

NameError: name 'pd' is not defined

In [75]:
df.head(1)

,page_id,title,link,fullcat,cat1,cat2,cat3
0,6632119,Gladstone Pottery Museum,www.stokemuseums.org.uk/visit/gpm,"['Culture.Visual arts.Architecture', 'Culture....",['Culture'],['Visual arts'],"['Architecture', 'Visual arts*']"


In [76]:
df.cat2

0                                           ['Visual arts']
1                                  ['Biography', 'Regions']
2                                                 ['Media']
3         ['Business and economics', 'Education', 'Regio...
4                                    ['Biography', 'Media']
                                ...                        
503051                               ['Biography', 'Media']
503052               ['Biography', 'Literature', 'Regions']
503053    ['Education', 'Politics and government', 'Regi...
503054                                          ['Regions']
503055                                        ['Biography']
Name: cat2, Length: 503056, dtype: object

In [77]:
allcat2 = pd.Series([c_ for c in df.cat2 for c_ in ast.literal_eval(c)])

In [17]:
allcat2

0                                     Visual arts
1                                BiographyRegions
2                                           Media
3          Business and economicsEducationRegions
4                                  BiographyMedia
                           ...                   
498216                             BiographyMedia
498217                 BiographyLiteratureRegions
498218    EducationPolitics and governmentRegions
498219                                    Regions
498220                                  Biography
Length: 498221, dtype: object

In [52]:
wiki_20_31cat = pd.DataFrame([])

nsamples = 20

for c in allcat2.unique():
    mask = df.cat2.apply(lambda x: c in x)
    df_c = merged[mask].sample(nsamples)[['page_id', 'link']]
    df_c['cat2'] = [re.sub(r' ', '_', c)] * nsamples
    wiki_20_31cat = pd.concat((wiki_20_31cat, df_c))

    

In [54]:
wiki_20_31cat

,page_id,link,cat2
375929,50649422,ffffound.com,Visual_arts
274038,4112508,www.fionabanner.com,Visual_arts
365016,46883492,www.museum.gov.lk/web/index.php?option=com_reg...,Visual_arts
491173,56672409,www.royalhawaiiancenter.com,Visual_arts
163614,53360924,gloriastollkarn.com,Visual_arts
...,...,...,...
50149,27327422,www.secmsg.emudhra.com,Mathematics
86370,46743384,peerio.com,Mathematics
64998,231712,www.cse-cst.gc.ca,Mathematics
135724,51849458,explorer.zcha.in,Mathematics


In [55]:
wiki_20_31cat.to_csv('/dlabdata1/lugeon/wiki_20_31cat.gz', compression='gzip')